## LINCS MOA prediciton for LINCS curated using decision trees

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing  import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [ ]:
file_prostate=pd.read_hdf("/opt/raid10/genomics/rashid/GCN/data/GSE92742_fully_restricted.hdf")




In [ ]:
df_prostate=file_prostate.reset_index(level=[0,1,2,3,4,5,6]) #change from multi index to single index

In [ ]:
prostate_y=df_prostate["moa"].values #change moa column to numpy array
df_prostate= df_prostate.drop(columns=["pert_id","pert_name","cell_id","primary_site","subtype","moa","Fold"])
prostate_X=df_prostate.values #get the matrix of feature values for 978 genes


In [ ]:
labels,uniques=pd.factorize(prostate_y) #labels are the encodings
prostate_y=labels

In [ ]:
sc=StandardScaler()
prostate_X=sc.fit_transform(prostate_X) #normalize feature data

In [ ]:
#SPLIT DATA
X_train,X_test,y_train,y_test=train_test_split(prostate_X,prostate_y,test_size=0.2,random_state=42)#split into train and test

In [ ]:
dct = DecisionTreeClassifier()
dct.fit(X_train,y_train)
pred_dct=dct.predict(X_test)  #1 min!

In [ ]:
#METRICS
print(confusion_matrix(y_test,pred_dct))
print(classification_report(y_test,pred_dct))
acc_dct=accuracy_score(y_test,pred_dct) #result for svm
print(acc_dct) #print precision score 
#results doesn't quite matches in the paper where accuracy is 53.2 ± 1.16, macro F1 is 32.6 ± 0.91. Took 4-5 minutes on macbook pro 2.7 GHz inter core i5 processor, 16 GB 1867 MHz DDR3 Memomry, Intel Iris Graphics 6100 1536 MB Graphics

In [ ]:
from sklearn.externals import joblib
joblib.dump(dct,'decisionTree_MOA_FullCurated_notTuned')

In [ ]:
dct_benchmark=DecisionTreeClassifier(min_impurity_decrease=0.0012257673197162205, min_samples_leaf= 2, min_weight_fraction_leaf=0.0020780796100258966, max_depth=10, max_features=None, criterion= 'entropy', max_leaf_nodes=None, min_samples_split=2, splitter='best')
dct_benchmark.fit(X_train,y_train)
pred_dct_benchmark=dct_benchmark.predict(X_test)  #1 min!

In [ ]:
#METRICS
print(confusion_matrix(y_test,pred_dct_benchmark))
print(classification_report(y_test,pred_dct_benchmark))
acc_dct_benchmark=accuracy_score(y_test,pred_dct_benchmark) #result for svm
print(acc_dct_benchmark) #print precision score 
#results almost matches in the paper where accuracy is 53.2 ± 1.16, macro F1 is 32.6 ± 0.91. Took 4-5 minutes on macbook pro 2.7 GHz inter core i5 processor, 16 GB 1867 MHz DDR3 Memomry, Intel Iris Graphics 6100 1536 MB Graphics

In [ ]:

joblib.dump(dct_benchmark,'decisionTree_MOA_FullCurated_Tuned')